In [3]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster
import numpy as np
from pathlib import Path

base = Path.cwd().parents[1]
df = pd.read_csv(base / "Data_clean/caract-2005-2023.csv")

df.head()


C:\Users\jeanl\AppData\Local\Temp\ipykernel_22396\3140086907.py:8: DtypeWarning: Columns (10,12,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(base / "Data_clean/caract-2005-2023.csv")


,Num_Acc,an,mois,jour,hrmn,lum,agg,int,atm,col,com,adr,gps,lat,long,dep,Accident_Id,date,weekday
0,2.005000e+11,2005,1,12,19:00,3,2,1,1.0,3.0,11,CD41B,M,50.515,2.944,590,NaN,2005-01-12,3
1,2.005000e+11,2005,1,21,16:00,1,2,1,1.0,1.0,51,rue de Lille,M,50.537,2.802,590,NaN,2005-01-21,5
2,2.005000e+11,2005,1,21,18:45,3,1,1,2.0,1.0,51,NaN,M,50.546,2.800,590,NaN,2005-01-21,5
3,2.005000e+11,2005,1,4,16:15,1,1,1,1.0,5.0,82,NaN,M,50.987,2.408,590,NaN,2005-01-04,2
4,2.005000e+11,2005,1,10,19:45,3,1,1,3.0,6.0,478,NaN,M,50.964,2.475,590,NaN,2005-01-10,1


In [ ]:
# Créer une colonne datetime complète avec date + heure
df["datetime"] = pd.to_datetime(
    df["an"].astype(str) + "-" +
    df["mois"].astype(str).str.zfill(2) + "-" +
    df["jour"].astype(str).str.zfill(2) + " " +
    df["hrmn"].astype(str),  # hrmn est déjà au format "19:00"
    format="%Y-%m-%d %H:%M"
)

# Vérifier le résultat
print("Exemple de datetime créées:")
print(df[["an", "mois", "jour", "hrmn", "datetime"]].head())

# Filtrer les accidents de nuit (exemple)
df_night = df[(df["datetime"].dt.hour >= 20) | (df["datetime"].dt.hour <= 6)]
print(f"\nAccidents de nuit (20h-6h): {len(df_night)} / {len(df)}")

In [10]:
df_night.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1231695 entries, 0 to 1231694
Data columns (total 19 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   Num_Acc      1176393 non-null  float64
 1   an           1231695 non-null  int64  
 2   mois         1231695 non-null  int64  
 3   jour         1231695 non-null  int64  
 4   hrmn         1231695 non-null  object 
 5   lum          1231695 non-null  int64  
 6   agg          1231695 non-null  int64  
 7   int          1231695 non-null  int64  
 8   atm          1231622 non-null  float64
 9   col          1231676 non-null  float64
 10  com          1231693 non-null  object 
 11  adr          1085797 non-null  object 
 12  gps          480052 non-null   object 
 13  lat          744627 non-null   float64
 14  long         741414 non-null   float64
 15  dep          1231695 non-null  object 
 16  Accident_Id  55302 non-null    float64
 17  date         1231695 non-null  object 
 18  we